# Вебинар 5. Домашнее задание

Само домашнее задание нахожится в конце ноутбука

---

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items

In [2]:
data = pd.read_csv('transaction_data.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
item_features = pd.read_csv('product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [4]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[879517, 934369, 1115576, 1124029, 5572301, 65..."
1,3,"[823704, 834117, 840244, 913785, 917816, 93870..."


In [5]:
n_items_before = data_train['item_id'].nunique()

data_train = prefilter_items(data_train)

n_items_after = data_train['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 90386 to 5001


/home/andreyl/Soft/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [6]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

user_item_matrix.head(3)

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15926885,15926886,15926887,15926927,15927033,15927403,15927661,15927850,16809471,17105257
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [8]:
user_item_matrix = bm25_weight(user_item_matrix.T).T  # Применяется к item-user матрице ! 

In [10]:
%%time

model = AlternatingLeastSquares(factors=20, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=4)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)


CPU times: user 17.7 s, sys: 61.8 ms, total: 17.8 s
Wall time: 4.28 s


-----

# Домашнее задание

1. Изучите структуру модуля src - **сделано**
2. Перенесите функции prefilter_items и postfilter_items из вебинара в модуль src.utils.py - **это уже было сделано**
3. Реализуйте функции get_similar_items_recommendation, get_similar_users_recommendation (они разбирались на вебинаре) и переместите в src.utils.py
4. Создайте модуль src.recommenders.py. Напищите код для класса ниже и положите его в src.recommenders.py
5. Проверьте, что все модули корректно импортируются
6. Если вы еще не прочитали [статью](https://habr.com/ru/company/hh/blog/347276/) о рекомендательных системах и поиске в hh.ru, то обязательно прочитайте

In [43]:
%%time
import scipy.spatial.distance as sc_dist
import operator

def get_similar_items_recommendation(user_item_matrix_,user, model, N=5):
    """Рекомендуем товары, похожие на топ-N купленных юзером товаров"""
    user_top = user_item_matrix_[user_item_matrix_.index==user].T[user_item_matrix_[user_item_matrix_.index==user].T[user]>0].index[:N]
#     print(user_top)
    top_similar_items=[]
    for i in user_top:
        sim_items = user_item_matrix_.T.iloc[user_item_matrix_.T.index==i,:]
        sim = 1-sc_dist.cdist(user_item_matrix_.T, sim_items, 'yule')
        top_similar_items=top_similar_items+sorted([[k,i[0]] for k,i in zip(user_item_matrix_.T.index,sim)], key=operator.itemgetter(1, 0))
    top_similar_items = [int(i) for i in list(np.array(sorted([[k,i[0]] for k,i in zip(user_item_matrix_.T.index,sim)], key=operator.itemgetter(1, 0))[-N:])[:,0])]
#     print(top_similar_items)
    return top_similar_items

def get_similar_users_recommendation(data_train_,user_item_matrix,user, model, N=5,n_users = 20):
    """Рекомендуем топ-N товаров, среди купленных похожими юзерами"""
    data_train_ = data_train_.groupby('user_id').mean()
    user_vector = data_train_.iloc[data_train_.index==user,:]
    sim = 1-sc_dist.cdist(data_train_, user_vector, 'cosine')
    top_similar_users = [k for k,i in sorted([[k,i[0]] for k,i in zip(user_item_matrix.index,sim)], key=operator.itemgetter(1, 0))]
    user_vector = user_item_matrix.loc[user_item_matrix.index==user,:]
    sim = 1-sc_dist.cdist(user_item_matrix, user_vector, 'cosine')
    top_similar_users = top_similar_users + [k for k,i in sorted([[k,i[0]] for k,i in zip(user_item_matrix.index,sim)], key=operator.itemgetter(1, 0))]
    res = [id_to_itemid[rec[0]] for rec in 
            model.recommend(userid=userid_to_id[user], 
                            user_items=csr_matrix(user_item_matrix.T[top_similar_users].T).tocsr(),   # на вход user-item matrix
                            N=N, 
                            filter_already_liked_items=False, 
                            filter_items=[itemid_to_id[999999]],
                            recalculate_user=True)]
#     print(res)
    if user%250==0:
        print(user/2500 * 100,'%')
    return res

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='sales_value', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )
user_item_matrix = user_item_matrix.astype(float)

CPU times: user 1.91 s, sys: 84.8 ms, total: 1.99 s
Wall time: 1.99 s


In [44]:
%%time
result['similar_items'] = result['user_id'].apply(lambda x: get_similar_items_recommendation(user_item_matrix,x, model=model, N=5))

CPU times: user 8min 27s, sys: 221 ms, total: 8min 27s
Wall time: 8min 29s


In [45]:
%%time
result['similar_items_top_users'] = result['user_id'].apply(lambda x: get_similar_users_recommendation(data_train,user_item_matrix,x, model=model, N=20))

10.0 %
30.0 %
40.0 %
50.0 %
60.0 %
80.0 %
90.0 %
100.0 %
CPU times: user 20min 20s, sys: 2min 25s, total: 22min 45s
Wall time: 21min 16s


In [46]:
result

,user_id,actual,similar_items,similar_items_top_users
0,1,"[879517, 934369, 1115576, 1124029, 5572301, 65...","[1075918, 6391176, 826695, 999999, 5850988]","[6534178, 1091365, 917033, 1107553, 480014, 10..."
1,3,"[823704, 834117, 840244, 913785, 917816, 93870...","[1426702, 2848087, 5716076, 5845857, 5850988]","[1104349, 1082185, 1006878, 972312, 960732, 93..."
2,5,"[913077, 1118028, 1386668]","[1119879, 1003421, 833715, 999999, 5850988]","[6534178, 480014, 1042942, 1091365, 397896, 10..."
3,6,"[825541, 859676, 999318, 1055646, 1067606, 108...","[731106, 1126711, 974766, 825749, 999999]","[5586642, 5583547, 5578494, 5584100, 5585966, ..."
4,7,"[929248, 948622, 1013572, 1022003, 1049892, 10...","[999631, 977374, 823176, 999999, 5850988]","[1107553, 1124729, 7441391, 1122268, 987480, 8..."
...,...,...,...,...
1986,2494,"[826265, 828393, 834303, 835236, 839147, 84036...","[1426702, 2848087, 5716076, 5845857, 5850988]","[5569471, 893501, 5569374, 1082185, 961554, 10..."
1987,2497,"[906777, 996833, 1032967, 15830980, 15928068, ...","[1045080, 980488, 1045571, 820361, 999999]","[1082185, 1126899, 960732, 1058997, 916993, 84..."
1988,2498,"[1014185, 13417441, 993638, 906335, 991948, 10...","[1426702, 823721, 999999, 5747233, 5850988]","[1106523, 1053690, 1124729, 1029743, 8090509, ..."
1989,2499,"[834117, 844441, 999971, 15924955, 18148096, 8...","[1045571, 1058939, 1095032, 821867, 999999]","[836445, 910673, 1103629, 1043590, 1025611, 97..."


In [47]:
result.apply(lambda row: precision_at_k(row['similar_items'], row['actual']), axis=1).mean(), result.apply(lambda row: precision_at_k(row['similar_items_top_users'], row['actual']), axis=1).mean()

(0.0163736815670517, 0.09864389753892455)

###### результаты, полученные при одновременном применении bm25 во всех вариантах
###### result.apply(lambda row: precision_at_k(row['bm25'], row['actual']), axis=1).mean(), result.apply(lambda row: precision_at_k(row['similar_items'], row['actual']), axis=1).mean(), result.apply(lambda row: precision_at_k(row['similar_items_top_users'], row['actual']), axis=1).mean()
###### (0.1884731290808639, 0.03787041687594194, 0.04218985434455071)

In [53]:
import pandas as pd
import numpy as np

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight


class MainRecommender:
    """Рекоммендации, которые можно получить из ALS
    
    Input
    -----
    user_item_matrix: pd.DataFrame
        Матрица взаимодействий user-item
    """
    
    def __init__(self, data, weighting=True):
        
        # your_code. Это не обязательная часть. Но если вам удобно что-либо посчитать тут - можно это сделать
        
        self.user_item_matrix = self.prepare_matrix(data)  # pd.DataFrame
        self.id_to_itemid, self.id_to_userid, self.itemid_to_id, self.userid_to_id = prepare_dicts(self.user_item_matrix)
        
        if weighting:
            self.user_item_matrix = bm25_weight(self.user_item_matrix.T).T 
        
        self.model = self.fit(self.user_item_matrix)
        self.own_recommender = self.fit_own_recommender(self.user_item_matrix)
     
    @staticmethod
    def prepare_matrix(data):
        
        user_item_matrix = pd.pivot_table(data, 
                                          index='user_id', columns='item_id', 
                                          values='sales_value', # Можно пробоват ьдругие варианты
                                          aggfunc='count', 
                                          fill_value=0
                                         )
        user_item_matrix = user_item_matrix.astype(float)
        
        return user_item_matrix
    
    @staticmethod
    def prepare_dicts(user_item_matrix):
        """Подготавливает вспомогательные словари"""
        
        userids = user_item_matrix.index.values
        itemids = user_item_matrix.columns.values

        matrix_userids = np.arange(len(userids))
        matrix_itemids = np.arange(len(itemids))

        id_to_itemid = dict(zip(matrix_itemids, itemids))
        id_to_userid = dict(zip(matrix_userids, userids))

        itemid_to_id = dict(zip(itemids, matrix_itemids))
        userid_to_id = dict(zip(userids, matrix_userids))
        
        return id_to_itemid, id_to_userid, itemid_to_id, userid_to_id
     
    @staticmethod
    def fit_own_recommender(user_item_matrix):
        """Обучает модель, которая рекомендует товары, среди товаров, купленных юзером"""
    
        own_recommender = ItemItemRecommender(K=1, num_threads=4)
        own_recommender.fit(csr_matrix(user_item_matrix).T.tocsr())
        
        return own_recommender
    
    @staticmethod
    def fit(user_item_matrix, n_factors=20, regularization=0.001, iterations=15, num_threads=4):
        """Обучает ALS"""
        
        model = AlternatingLeastSquares(factors=factors, 
                                             regularization=regularization,
                                             iterations=iterations,  
                                             num_threads=num_threads)
        model.fit(csr_matrix(self.user_item_matrix).T.tocsr())
        
        return model

    def get_similar_items_recommendation(self, user, N=5):
        """Рекомендуем товары, похожие на топ-N купленных юзером товаров"""
        user_top = user_item_matrix_[user_item_matrix_.index==user].T[user_item_matrix_[user_item_matrix_.index==user].T[user]>0].index[:N]
    #     print(user_top)
        top_similar_items=[]
        for i in user_top:
            sim_items = user_item_matrix_.T.iloc[user_item_matrix_.T.index==i,:]
            sim = 1-sc_dist.cdist(user_item_matrix_.T, sim_items, 'yule')
            top_similar_items=top_similar_items+sorted([[k,i[0]] for k,i in zip(user_item_matrix_.T.index,sim)], key=operator.itemgetter(1, 0))
        top_similar_items = [int(i) for i in list(np.array(sorted([[k,i[0]] for k,i in zip(user_item_matrix_.T.index,sim)], key=operator.itemgetter(1, 0))[-N:])[:,0])]
    #     print(top_similar_items)

        assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
        return res
    
#     def get_similar_users_recommendation(self, user, N=5):
#     """Рекомендуем топ-N товаров, среди купленных похожими юзерами"""
    
    def get_similar_users_recommendation(user_item_matrix,user, model, N=5,n_users = 20):
        """Рекомендуем топ-N товаров, среди купленных похожими юзерами"""
        user_vector = user_item_matrix.iloc[user_item_matrix.index==user,:]
        sim = 1-sc_dist.cdist(user_item_matrix, user_vector, 'cosine')
        top_similar_users = [k for k,i in sorted([[k,i[0]] for k,i in zip(user_item_matrix.index,sim)], key=operator.itemgetter(1, 0))]
        res = [id_to_itemid[rec[0]] for rec in 
                model.recommend(userid=userid_to_id[user], 
                                user_items=csr_matrix(user_item_matrix.T[top_similar_users].T).tocsr(),   # на вход user-item matrix
                                N=N, 
                                filter_already_liked_items=False, 
                                filter_items=[itemid_to_id[999999]],
                                recalculate_user=True)]

        assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
        return res

----

Проверка, что все работает

In [146]:
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recs2 import MainRecommender
import scipy.spatial.distance as scdist
import operator

In [147]:
a = data_train[data_train.index==0]
b = user_item_matrix[user_item_matrix.index==1]
b

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15926885,15926886,15926887,15926927,15927033,15927403,15927661,15927850,16809471,17105257
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0


In [140]:
x = MainRecommender.get_similar_users_recommendation(data_train,1, model=model, N=5)
x

[1062128, 1009449, 1075368, 908846, 850044]

In [148]:
x = MainRecommender.get_similar_items_recommendation(data_train,1, model=model, N=5)
x

[[ 0.50520416]
 [ 0.13471822]
 [ 0.33975388]
 ...
 [ 0.33789712]
 [-0.08258759]
 [ 0.20511096]]
[[-1.        ]
 [ 0.45300593]
 [-1.        ]
 ...
 [ 0.57312321]
 [ 0.11495591]
 [ 0.42856392]]
[[ 0.72858287]
 [-0.0895117 ]
 [ 0.22779043]
 ...
 [ 0.37328094]
 [ 0.1253406 ]
 [-0.28114827]]
[[ 0.74379504]
 [ 0.09240246]
 [-1.        ]
 ...
 [ 0.01170408]
 [ 0.49594866]
 [ 0.33647364]]
[[-1.        ]
 [ 0.37922987]
 [-1.        ]
 ...
 [ 0.32785026]
 [-0.10469174]
 [ 0.2062919 ]]


[1075918, 6391176, 826695, 999999, 5850988]